# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846854218428                   -0.70    4.8         
  2   -7.852317293529       -2.26       -1.53    1.0   25.4ms
  3   -7.852617131640       -3.52       -2.57    1.5   28.5ms
  4   -7.852646034871       -4.54       -2.92    2.8   35.0ms
  5   -7.852646534303       -6.30       -3.24    1.2   26.6ms
  6   -7.852646680170       -6.84       -4.25    1.2   26.8ms
  7   -7.852646686667       -8.19       -5.02    2.2   34.5ms
  8   -7.852646686726      -10.23       -5.54    1.8   30.3ms
  9   -7.852646686729      -11.57       -5.74    1.2   27.6ms
 10   -7.852646686730      -12.02       -6.82    1.0   25.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846914142300                   -0.70           4.8         
  2   -7.852528041758       -2.25       -1.64   0.80    2.0    277ms
  3   -7.852635803278       -3.97       -2.72   0.80    1.0   23.5ms
  4   -7.852646451979       -4.97       -3.24   0.80    2.2   30.8ms
  5   -7.852646665545       -6.67       -4.05   0.80    1.0   23.5ms
  6   -7.852646686321       -7.68       -4.68   0.80    1.8   28.4ms
  7   -7.852646686707       -9.41       -5.43   0.80    1.5   26.9ms
  8   -7.852646686729      -10.67       -6.48   0.80    1.8   27.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.391972e+01     3.620439e+00
 * time: 0.06254792213439941
     1     1.204049e+00     1.757762e+00
 * time: 0.30686402320861816
     2    -1.293861e+00     1.989219e+00
 * time: 0.3320930004119873
     3    -3.670805e+00     1.745091e+00
 * time: 0.3679769039154053
     4    -4.657833e+00     1.597114e+00
 * time: 0.4038560390472412
     5    -6.489232e+00     9.671444e-01
 * time: 0.43950390815734863
     6    -7.183390e+00     4.732329e-01
 * time: 0.47556304931640625
     7    -7.531553e+00     3.145605e-01
 * time: 0.5003259181976318
     8    -7.680524e+00     1.514857e-01
 * time: 0.5257258415222168
     9    -7.756214e+00     1.340858e-01
 * time: 0.5508320331573486
    10    -7.794178e+00     8.999203e-02
 * time: 0.5761868953704834
    11    -7.821973e+00     5.812033e-02
 * time: 0.6017320156097412
    12    -7.840898e+00     5.548180e-02
 * time: 0.6277649402618408
    13    -7.848719e+00     2.248437e-02
 * time: 0.652943

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846671660661                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645936411                   -1.64         
  2   -7.852646686730       -6.12       -3.72    2.18s
  3   -7.852646686730      -13.34       -7.27    178ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.1103556136645555e-7
|ρ_newton - ρ_scfv| = 8.485022497642605e-7
|ρ_newton - ρ_dm|   = 8.435599541715113e-10
